# Run Random Forest

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import sys

sys.path.insert(0, "../../src")
from run_mp import *
from nfi_wrangling import *

In [24]:
nfi_data = pd.read_csv(
    "../00_process_nfi_data/20231108-211512_nfi_dataset_for_analysis copy.csv",
    index_col=[0],
)

gee_data = pd.read_feather("../02_process_gee_data/final_gee_predictor_dataset.feather")
site_dict = pd.read_csv("../01_download_raw_gee_data/sites_years.csv")
site_dict = site_dict.rename(
    columns={"id": "SiteID", "x": "lon", "y": "lat", "first_visit": "first_year"}
)

/var/folders/39/534x_msj7sl1kcxj2mg109x40000gn/T/ipykernel_67343/3610276152.py:1: DtypeWarning: Columns (51,58,61,73,74,75,88,89,90,91,106,107,112,113,115,119,125,127,156) have mixed types. Specify dtype option on import or set low_memory=False.
  nfi_data = pd.read_csv(


In [42]:
gee_data = gee_data.merge(
    site_dict, on=["SiteID", "first_year", "lon", "lat"], how="left"
)
gee_data

KeyError: 'lon'

In [3]:
df_test = nfi_data.query("idp == 511880").copy()
# [print(col) for col in df_test.columns]
df_test.insert(0, "tree_state_change", df_test.pop("tree_state_change"))
df_test

,tree_state_change,idp,perc_cut,dominant_species,dominant_tree_class,a,tree_id,visit_1,visit_2,revisit_state,...,obsriv2,typriv,distriv,denivriv,lign2,lign1,herb,mousse,obsveget,obsdate
4105,alive_dead,511880,0.866859,Pinus,pinus,4,511880_4,2010,2015,revisited,...,NaN,NaN,NaN,NaN,7.0,0.0,10.0,8.0,NaN,NaN


In [7]:
calculate_growth_mortality(df_test)

,idp,n_ini,n_sur,n_fin,ba_at_v1_of_alive_trees,ba_at_v2_of_alive_trees,ba_at_v1_of_survivors,ba_at_v2_of_survivors,ba_at_v2_of_dead,ba_loss_yr,ba_gain_yr,ba_ingr_yr,ba_growth_abs,ba_growth_rate,ba_loss_abs,ba_loss_rate,ba_change_abs,ba_change_rel
0,511880,1,0,0,1.18181,0.0,0.0,0.0,1.508064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.236362,-0.04


In [8]:
from tqdm import tqdm

# Assuming your DataFrame is named nfi_data
grouped_data = nfi_data.groupby("idp")


# Define a function to apply to each group
def process_group(group):
    result = calculate_growth_mortality(group)
    return result


# Apply the function to each group and concatenate the results
result_df = pd.concat([process_group(group) for _, group in tqdm(grouped_data)])

# Reset the index to make the output cleaner
result_df.reset_index(drop=True, inplace=True)

# Display the result
# print(result_df)

100%|██████████| 38696/38696 [06:30<00:00, 99.09it/s] 


           idp  n_ini  n_sur  n_fin  ba_at_v1_of_alive_trees  \
0       500002     10     10     10                88.080937   
1       500008      7      7      7                 3.856284   
2       500012     11     11     11                45.807307   
3       500013     11     10     10                41.782830   
4       500042      5      5      5                49.129021   
...        ...    ...    ...    ...                      ...   
38691  1131409     13     13     13                96.011954   
38692  1131410     11      8      8                20.160079   
38693  1131419     14     14     14                 8.014100   
38694  1131424     16     16     16                73.638615   
38695  1131437     17     17     17                34.879210   

       ba_at_v2_of_alive_trees  ba_at_v1_of_survivors  ba_at_v2_of_survivors  \
0                    94.781104              88.080937              94.781104   
1                     5.790708               3.856284               5.7

In [10]:
result_df.to_feather("nfi-idp_level_aggregated_-growth_mortality.feather")
result_df

,idp,n_ini,n_sur,n_fin,ba_at_v1_of_alive_trees,ba_at_v2_of_alive_trees,ba_at_v1_of_survivors,ba_at_v2_of_survivors,ba_at_v2_of_dead,ba_loss_yr,ba_gain_yr,ba_ingr_yr,ba_growth_abs,ba_growth_rate,ba_loss_abs,ba_loss_rate,ba_change_abs,ba_change_rel
0,500002,10,10,10,88.080937,94.781104,88.080937,94.781104,0.000000,0.000000,1.466279,1.466279,1.340033,1.521366,0.000000,0.000000,77.164917,0.175214
1,500008,7,7,7,3.856284,5.790708,3.856284,5.790708,0.000000,0.000000,8.131010,8.131010,0.386885,10.032580,0.000000,0.000000,5.019451,0.260326
2,500012,11,11,11,45.807307,50.992115,45.807307,50.992115,0.000000,0.000000,2.144548,2.144548,1.036962,2.263747,0.000000,0.000000,41.830653,0.182637
3,500013,11,10,10,41.782830,43.375193,40.686940,43.375193,1.095890,0.531567,1.279611,1.279611,0.537651,1.321433,0.219178,0.492855,35.018627,0.167622
4,500042,5,5,5,49.129021,53.249145,49.129021,53.249145,0.000000,0.000000,1.610637,1.610637,0.824025,1.677267,0.000000,0.000000,43.423341,0.176773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38691,1131409,13,13,13,96.011954,107.117760,96.011954,107.117760,0.000000,0.000000,2.189122,2.189122,2.221161,2.313422,0.000000,0.000000,87.915370,0.183134
38692,1131410,11,8,8,20.160079,10.426242,9.489742,10.426242,10.781125,15.069858,1.882289,1.882289,0.187300,1.973709,2.156225,10.167340,6.394226,0.063435
38693,1131419,14,14,14,8.014100,12.749921,8.014100,12.749921,0.000000,0.000000,9.286451,9.286451,0.947164,11.818722,0.000000,0.000000,11.147101,0.278187
38694,1131424,16,16,16,73.638615,81.607097,73.638615,81.607097,0.000000,0.000000,2.054934,2.054934,1.593696,2.164213,0.000000,0.000000,66.879374,0.181642


In [38]:
# Merge with nfi data
df_tmp = (
    nfi_data[["idp", "lat", "lon", "visit_1"]]
    .rename(columns={"visit_1": "first_year"})
    .drop_duplicates()
    .merge(result_df, on="idp", how="right")
)
df_tmp
# idp_level = df_tmp.merge(gee_data, on=["first_year", "lat", "lon"], how="left")
# idp_level

,idp,lat,lon,first_year,n_ini,n_sur,n_fin,ba_at_v1_of_alive_trees,ba_at_v2_of_alive_trees,ba_at_v1_of_survivors,...,ba_at_v2_of_dead,ba_loss_yr,ba_gain_yr,ba_ingr_yr,ba_growth_abs,ba_growth_rate,ba_loss_abs,ba_loss_rate,ba_change_abs,ba_change_rel
0,500002,48.253411,7.378014,2010,10,10,10,88.080937,94.781104,88.080937,...,0.000000,0.000000,1.466279,1.466279,1.340033,1.521366,0.000000,0.000000,77.164917,0.175214
1,500008,48.050171,6.069031,2010,7,7,7,3.856284,5.790708,3.856284,...,0.000000,0.000000,8.131010,8.131010,0.386885,10.032580,0.000000,0.000000,5.019451,0.260326
2,500012,49.139978,5.736655,2010,11,11,11,45.807307,50.992115,45.807307,...,0.000000,0.000000,2.144548,2.144548,1.036962,2.263747,0.000000,0.000000,41.830653,0.182637
3,500013,49.200153,1.132647,2010,11,10,10,41.782830,43.375193,40.686940,...,1.095890,0.531567,1.279611,1.279611,0.537651,1.321433,0.219178,0.492855,35.018627,0.167622
4,500042,49.187090,2.860154,2010,5,5,5,49.129021,53.249145,49.129021,...,0.000000,0.000000,1.610637,1.610637,0.824025,1.677267,0.000000,0.000000,43.423341,0.176773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38691,1131409,42.686748,2.156438,2016,13,13,13,96.011954,107.117760,96.011954,...,0.000000,0.000000,2.189122,2.189122,2.221161,2.313422,0.000000,0.000000,87.915370,0.183134
38692,1131410,47.899971,7.424664,2016,11,8,8,20.160079,10.426242,9.489742,...,10.781125,15.069858,1.882289,1.882289,0.187300,1.973709,2.156225,10.167340,6.394226,0.063435
38693,1131419,48.851908,-3.221877,2016,14,14,14,8.014100,12.749921,8.014100,...,0.000000,0.000000,9.286451,9.286451,0.947164,11.818722,0.000000,0.000000,11.147101,0.278187
38694,1131424,44.924973,3.757202,2016,16,16,16,73.638615,81.607097,73.638615,...,0.000000,0.000000,2.054934,2.054934,1.593696,2.164213,0.000000,0.000000,66.879374,0.181642


In [6]:
grouped = nfi_data.groupby("idp", as_index=False)
df_list = [group for name, group in grouped]

out = run_mp(
    calculate_growth_mortality,
    df_list,
    combine_func=pd.concat,
    progress_bar=True,
    num_cores=10,
)

  0%|          | 0/38696 [00:46<?, ?it/s] 98.01it/s]


KeyboardInterrupt: 